In [20]:
import random
import torch

In [21]:
# chargement des données

fichier = open('villes.txt')
donnees = fichier.read()
villes = donnees.replace('\n', ',').split(',')

# préparation des données

# on rajoute le token . au début et en fin (fait office de signal de départ et de fin)
for ville, i in zip(villes, range(len(villes))):
    villes[i] = '.' + ville + '.'

# création du vocabulaire
vocabulaire = []

for ville in villes:
    for c in ville:
        if c not in vocabulaire:
            vocabulaire.append(c)

vocabulaire = sorted(vocabulaire)
vocabulaire[0] = '.' # 0 est " " et 3 est "." -> on échange
vocabulaire[3] = ' '

# pour convertir char <-> int
char_to_int = {}
int_to_char = {}

for (c, i) in zip(vocabulaire, range(len(vocabulaire))):
    char_to_int[c] = i
    int_to_char[i] = c

# random (0-gram)

Aucun apprentissage : on génère des lettres du vocabulaire de manière aléatoire.

In [4]:
''.join(random.choice(vocabulaire) for _ in range(30))

"âpjfpüzvnüyâlvk'dq pqtj-'ûexgm"

C'est pas terrible...

In [6]:
# on calcule facilement le coût
# puisqu'on choisit aléatoirement les lettres, cela revient à assigner une proba de 1/len(voc) à toutes les lettres
# la formule du cout est -log p_model(bonne lettre | contexte)
# ici le contexte est vide, et on obtient donc -log(1/len(voca))

-torch.log(torch.tensor(1/len(vocabulaire)))

tensor(3.7842)

# unigram

Cette fois, il y a un tout petit "apprentissage" : on va compter la fréquence de chaque lettre parmis les 36000 noms de communes.

Ensuite, on génère des lettres à partir de ces fréquences.

In [8]:
# création du dataset

# matrice des données, qui sera de taille (M, 1) pour le unigram
# on y place simplement toutes les lettres de toutes les communes
X = []

for ville in villes:
    for char in ville:
        X.append([char_to_int[char]])

X = torch.asarray(X) # (M, 1)

In [9]:
# modèle uni-gram
P = torch.zeros((len(vocabulaire))) # liste de probabilités d'apparition de chaque lettre

for i in range(X.shape[0]):
    P[X[i]] += 1 # on augmente le compteur de chaque lettre rencontrée

P = P / P.sum(dim=0, keepdim=True) # on transforme les nombres d'apparitions en probabilités

In [11]:
g = torch.Generator().manual_seed(40+5)

for _ in range(10):
    # on génère une lettre tant qu'on ne tombe pas sur ".", qui signifie la fin
    nom = "."
    while nom[-1] != "." or len(nom) == 1:
        next_char = int_to_char[torch.multinomial(P, num_samples=1, replacement=True, generator=g).item()]
        nom = nom + next_char
    print(nom[1:-1])


a-cqsu
n
o
sayoemnraro

ianèriaesom
aom--ei
l
oitabu


In [12]:
# calcul du coût
# cette fois, on parcours toutes les données
# rappel de la formule : moyenne(log p_modele(lettre suivante | contexte))
# la moyenne se fait sur l'entièreté d'un nom de commune, et sur l'ensemble des noms de communes

nll = 0
for i in range(X.shape[0]):
    nll += torch.log(P[X[i, 0]]) # log p_modele(lettre) (contexte vide ici)
-nll/X.shape[0] # moyenne

tensor(2.9820)

# bigrams

In [109]:
# création du dataset

# matrice des données, qui sera de taille (M, 2) pour le bigram
# on y place les lettres deux par deux 
# par exemple, à partir de paris, on construirait les exemples ".p", "pa", "ar", "ri", "is", "s."

X = []

for ville in villes:
    for ch1, ch2 in zip(ville, ville[1:]):
        X.append([char_to_int[ch1], char_to_int[ch2]])

X = torch.asarray(X) # (M, 2)

In [110]:
# modèle bigram
P = torch.zeros((len(vocabulaire), len(vocabulaire))) # liste de probabilités d'apparition de chaque couple

for i in range(X.shape[0]):
    P[X[i, 0], X[i, 1]] += 1 # on augmente le compteur de chaque couple rencontré

P = P / P.sum(dim=1, keepdim=True) # on divise pour obtenir des probabilitiés
# la dimension 0 correspond à la première lettre de chaque couple (le contexte)
# la dimension 1 à la seconde lettre (la lettre prédite)

# par exemple, P[char_to_int['a']] correspond à 45 nombres, la distribution de probabilités sur les lettres qui suivent le a

In [112]:
g = torch.Generator().manual_seed(42+4)

for _ in range(10):
    nom = "."
    while nom[-1] != "." or len(nom) == 1:
        last_char = nom[-1]
        next_char = int_to_char[torch.multinomial(P[char_to_int[last_char]], num_samples=1, replacement=True, generator=g).item()]
        nom = nom + next_char
    print(nom[1:-1])

layst
stl
pin-s-d-sus
chisspen
moue c-gu-main-peu
llergnch
cheusollet
lle
bllaint-lenévir-sesas-penaiemey
norgitagesa


In [12]:
# calcul du coût
# rappel de la formule : moyenne(log p_modele(lettre suivante | contexte))
# la moyenne se fait sur l'entièreté d'un nom de commune, et sur l'ensemble des noms de communes

nll = 0
for i in range(X.shape[0]):
    nll += torch.log(P[X[i, 0], X[i, 1]]) # log p_modele(lettre | contexte) avec contexte = X[i, 0], lettre = X[i, 1]
-nll/X.shape[0]

tensor(2.3906)

In [14]:
f"Nombre de paramètres : {len(vocabulaire)*len(vocabulaire)}"

'Nombre de paramètres : 1936'

# trigrams

In [22]:
# on rajoute le token . au début et en fin
for ville, i in zip(villes, range(len(villes))):
    villes[i] = '.' + ville + "."

In [23]:
# création du dataset

X = [] # taille (M, 3) pour le trigram

# pour paris, on construit "..p", ".pa", "par", "ari", "ris", "is.", "s.."
# d'où la nécessite de rajouter un . au début et à la fin

for ville in villes:
    for ch1, ch2, ch3 in zip(ville, ville[1:], ville[2:]):
        X.append([char_to_int[ch1], char_to_int[ch2], char_to_int[ch3]])

X = torch.asarray(X) # (M, 3)

In [24]:
# modèle trigram
P = torch.zeros((len(vocabulaire), len(vocabulaire), len(vocabulaire))) # une proba pour chaque trio de lettre

for i in range(X.shape[0]):
    P[X[i, 0], X[i, 1], X[i, 2]] += 1

P = P / P.sum(dim=2, keepdim=True) # la dernière dimension correspond à la prochain lettre, les 2 premières aux lettres de contexte

In [25]:
g = torch.Generator().manual_seed(4354)

for _ in range(10):
    nom = ".."
    while nom[-1] != "." or len(nom) == 2:
        char_moins_1 = nom[-1]
        char_moins_2 = nom[-2]

        next_char = int_to_char[torch.multinomial(P[char_to_int[char_moins_2], char_to_int[char_moins_1]], num_samples=1, replacement=True, generator=g).item()]
        nom = nom + next_char

    print(nom[2:-1])

igorville
sérômes-ne
pes
mille
carren-d'as
quintotse
print-méneux
lournac
la-venoisille pelle
gace


In [26]:
# loss
nll = 0
for i in range(X.shape[0]):
    nll += torch.log(P[X[i, 0], X[i, 1], X[i, 2]]) # log p_modele(lettre | contexte)
-nll/X.shape[0]

tensor(1.8118)

In [30]:
f"Nombre de paramètres : {len(vocabulaire)**3}"

'Nombre de paramètres : 85184'